In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
df = pd.read_csv('testdata.csv')
df.head(2)

In [ ]:
df['label_goldstd'].value_counts()

In [ ]:
df.columns

In [ ]:
df['preds'] = df['preds'].astype(str)
df['label_goldstd'] = df['label_goldstd'].astype(str)

In [ ]:
type(df['preds'][1])

In [ ]:
type(df['label_goldstd'][1])

In [ ]:
df['preds'] = pd.to_numeric(df['preds'])
df['label_goldstd'] = pd.to_numeric(df['label_goldstd'])

In [ ]:
df1 = df[df['label_goldstd'] == 1].copy()  
df0 = df[df['label_goldstd'] == 0].copy()  

In [ ]:
df1['label_goldstd'].value_counts()

In [ ]:
df0['label_goldstd'].value_counts()

# For Labels

In [ ]:
df.head(2)

In [ ]:
# if 2 labels, update accordingly 

def match_label(gold, pred):
    if gold == 1 and pred == 0:
        return "FN"
    elif pred == gold == 1:
        return "TP"
    elif pred == gold == 0:
        return "TN"
    elif gold == 0 and pred == 1:
        return "FP"
    else:
        return "no match"

df['label_match'] = df.apply(lambda row: match_label(row['label_goldstd'], row['preds']), axis=1)
df1['label_match'] = df1.apply(lambda row: match_label(row['label_goldstd'], row['preds']), axis=1)
df0['label_match'] = df0.apply(lambda row: match_label(row['label_goldstd'], row['preds']), axis=1)

In [ ]:
df['label_match'].value_counts()

In [ ]:
df1['label_match'].value_counts()

In [ ]:
df0['label_match'].value_counts()

In [ ]:
count_label_metrics = df['label_match'].value_counts()
count_label_metrics_1 = df1['label_match'].value_counts()
count_label_metrics_0 = df0['label_match'].value_counts()

In [ ]:
TP = count_label_metrics.get('TP', 0)
FN = count_label_metrics.get('FN', 0)
FP = count_label_metrics.get('FP', 0)
TN = count_label_metrics.get('TN', 0)

print("TP: ", TP)
print("FN: ", FN)
print("FP: ", FP)
print("TN: ", TN)

In [ ]:
precision = TP/(TP+FP)
recall = TP/(TP+FN)
f1 = 2*(precision*recall)/(precision+recall)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)

In [ ]:
SE_precision = math.sqrt((precision*(1-precision)/(TP+FP)))
SE_recall = math.sqrt((recall*(1-recall)/(TP+FN)))

print("Precision std error: ", SE_precision)
print("Recall std error: ", SE_recall)

In [ ]:
z = 1.96

In [ ]:
precision_lower_ci = precision - (1.96*SE_precision)
precision_upper_ci = precision + (1.96*SE_precision)

recall_lower_ci = recall - (1.96*SE_recall)
recall_upper_ci = recall + (1.96*SE_recall)

f1_lower_ci = 2*(precision_lower_ci*recall_lower_ci)/(precision_lower_ci+recall_lower_ci)
f1_upper_ci = 2*(precision_upper_ci*recall_upper_ci)/(precision_upper_ci+recall_upper_ci)

print("Precision lower CI: ", precision_lower_ci)
print("Precision upper CI: ", precision_upper_ci)
print("Recall lower CI: ", recall_lower_ci)
print("Recall lower CI: ", recall_upper_ci)
print("F1 lower CI: ", f1_lower_ci)
print("F1 lower CI: ", f1_upper_ci)

In [ ]:
model_label_metrics = {
    'Metric': ['F1-score', 'Precision', 'Recall'],
    'Value': [f1, precision, recall],
    'Lower 95% CI': [f1_lower_ci, precision_lower_ci, recall_lower_ci],
    'Upper 95% CI': [f1_upper_ci, precision_upper_ci, recall_upper_ci]
}

In [ ]:
df_label_metrics = pd.DataFrame(model_label_metrics)
df_label_metrics

In [ ]:
df_label_metrics.to_csv('manual_eval_metrics_jun26model.csv')

In [ ]:

 
# Function to match label for class 1 and class 0
def match_label(gold, pred):
    if gold == 1 and pred == 0:
        return "FN_1"  # False Negative for class 1
    elif pred == gold == 1:
        return "TP_1"  # True Positive for class 1
    elif pred == gold == 0:
        return "TN_0"  # True Negative for class 0
    elif gold == 0 and pred == 1:
        return "FP_1"  # False Positive for class 1
    else:
        return "no match"
 
# Apply match_label to the dataframe
df['label_match'] = df.apply(lambda row: match_label(row['label_goldstd'], row['preds']), axis=1)
 
# Count the occurrences of each label match (FN, FP, TP, TN)
count_label_metrics = df['label_match'].value_counts()
 
# Extract metrics for class 1
TP_1 = count_label_metrics.get('TP_1', 0)
FN_1 = count_label_metrics.get('FN_1', 0)
FP_1 = count_label_metrics.get('FP_1', 0)
TN_0 = count_label_metrics.get('TN_0', 0)
 
# Precision, Recall, F1 for class 1 (positive class)
precision_1 = TP_1 / (TP_1 + FP_1) if (TP_1 + FP_1) > 0 else 0
recall_1 = TP_1 / (TP_1 + FN_1) if (TP_1 + FN_1) > 0 else 0
f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0
 
# Extract metrics for class 0 (negative class)
TP_0 = TN_0  # True negatives for class 0 are equivalent to TN_0
FP_0 = FN_1  # False negatives for class 1 are false positives for class 0
FN_0 = FP_1  # False positives for class 1 are false negatives for class 0
TN_1 = TP_1  # True positives for class 1 are true negatives for class 0
 
# Precision, Recall, F1 for class 0 (negative class)
precision_0 = TP_0 / (TP_0 + FP_0) if (TP_0 + FP_0) > 0 else 0
recall_0 = TP_0 / (TP_0 + FN_0) if (TP_0 + FN_0) > 0 else 0
f1_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0) if (precision_0 + recall_0) > 0 else 0
 
# Standard error calculations for both classes
SE_precision_1 = math.sqrt((precision_1 * (1 - precision_1)) / (TP_1 + FP_1)) if (TP_1 + FP_1) > 0 else 0
SE_recall_1 = math.sqrt((recall_1 * (1 - recall_1)) / (TP_1 + FN_1)) if (TP_1 + FN_1) > 0 else 0
 
SE_precision_0 = math.sqrt((precision_0 * (1 - precision_0)) / (TP_0 + FP_0)) if (TP_0 + FP_0) > 0 else 0
SE_recall_0 = math.sqrt((recall_0 * (1 - recall_0)) / (TP_0 + FN_0)) if (TP_0 + FN_0) > 0 else 0
 
# Confidence intervals for both classes
precision_1_lower_ci = precision_1 - (1.96 * SE_precision_1)
precision_1_upper_ci = precision_1 + (1.96 * SE_precision_1)
 
recall_1_lower_ci = recall_1 - (1.96 * SE_recall_1)
recall_1_upper_ci = recall_1 + (1.96 * SE_recall_1)
 
precision_0_lower_ci = precision_0 - (1.96 * SE_precision_0)
precision_0_upper_ci = precision_0 + (1.96 * SE_precision_0)
 
recall_0_lower_ci = recall_0 - (1.96 * SE_recall_0)
recall_0_upper_ci = recall_0 + (1.96 * SE_recall_0)
 
# F1 confidence intervals for class 1
f1_1_lower_ci = 2 * (precision_1_lower_ci * recall_1_lower_ci) / (precision_1_lower_ci + recall_1_lower_ci) if (precision_1_lower_ci + recall_1_lower_ci) > 0 else 0
f1_1_upper_ci = 2 * (precision_1_upper_ci * recall_1_upper_ci) / (precision_1_upper_ci + recall_1_upper_ci) if (precision_1_upper_ci + recall_1_upper_ci) > 0 else 0
 
# F1 confidence intervals for class 0
f1_0_lower_ci = 2 * (precision_0_lower_ci * recall_0_lower_ci) / (precision_0_lower_ci + recall_0_lower_ci) if (precision_0_lower_ci + recall_0_lower_ci) > 0 else 0
f1_0_upper_ci = 2 * (precision_0_upper_ci * recall_0_upper_ci) / (precision_0_upper_ci + recall_0_upper_ci) if (precision_0_upper_ci + recall_0_upper_ci) > 0 else 0
 
# Create a DataFrame for metrics
model_label_metrics = {
    'Metric': ['F1-score Class 1', 'Precision Class 1', 'Recall Class 1',
               'F1-score Class 0', 'Precision Class 0', 'Recall Class 0'],
    'Value': [f1_1, precision_1, recall_1, f1_0, precision_0, recall_0],
    'Lower 95% CI': [f1_1_lower_ci, precision_1_lower_ci, recall_1_lower_ci, f1_0_lower_ci, precision_0_lower_ci, recall_0_lower_ci],
    'Upper 95% CI': [f1_1_upper_ci, precision_1_upper_ci, recall_1_upper_ci, f1_0_upper_ci, precision_0_upper_ci, recall_0_upper_ci]
}
 
df_label_metrics = pd.DataFrame(model_label_metrics)
print(df_label_metrics)


In [ ]:
df_label_metrics.to_csv('manual_eval_metrics_classes_jun26model.csv')